# Client

In [1]:
import socket, os
from threading import Thread

##Client
class Client:
    
    def __init__(self, port):
        self.__port = port
        self.__outgoingsocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.__incomingsocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.__outgoingsocket.connect(('localhost', self.__port))
    
    def __displayMainMenu(self):
        print(' Client Messaging')
        print('___________________')
        print('1. Login')
        print('2. Create User')
        print('3. Close')
        
    def runClient(self):
        self.__displayMainMenu()
        choice = input(':')
        while choice != '3':
            if choice == '1':
                self.__login()
            elif choice == '2':
                self.__createNewUser()
            else:
                print('Invalid Command')
            self.__displayMainMenu()
            choice = input(':')
            self.__outgoingsocket.close()
            self.__incomingsocket.close()
            self.__outgoingsocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.__incomingsocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.__outgoingsocket.connect(('localhost', self.__port))
                
    def __sendMsg(self, msg, connection):
        connection.send(msg.encode())
        msg = connection.recv(1024).decode()
        arguments = msg.strip().split('|')
        return arguments
    
    def __login(self):
        os.system('clear')
        phone = input('Phone: ')
        pwd = input('Password: ')
        resp = self.__sendMsg('L|'+phone+'|'+pwd, self.__outgoingsocket)
        if resp[0] == '0':
            self.__incomingsocket.connect(('localhost', self.__port+1))
            resp = self.__sendMsg('C|'+phone+'|'+pwd, self.__incomingsocket)
            if resp[0] == '0':
                Thread(target=self.__recvMsgs).start()
                self.__displayMessagingInterface(phone, pwd)
            else:
                print('Error: '+resp[1])
        else:
            print('Error: '+resp[1])
                    
    def __createNewUser(self):
        os.system('clear')
        phone = input('Phone: ')
        pwd = input('Password: ')
        resp = self.__sendMsg('A|'+phone+'|'+pwd, self.__outgoingsocket)
        if resp[0] == '0': 
            self.__incomingsocket.connect(('localhost', self.__port+1))
            resp = self.__sendMsg('C|'+phone+'|'+pwd, self.__incomingsocket)
            if resp[0] == '0':
                Thread(target=self.__recvMsgs).start()
                self.__displayMessagingInterface(phone, pwd)
            else:
                print('Error: '+resp[1])
        else:
            print('Error: '+resp[1])
                    
    def __displayMessagingInterface(self, phone, pwd):
        print('To send msg, use format M|<recipient number>|<msg>')
        print('Type QUIT to quit')
        msg = input(':')
        while msg != 'QUIT':
            arguments = msg.strip().split('|')
            msg = arguments[0]+'|'+phone+'|'+pwd+'|'+arguments[1]+'|'+arguments[2]
            resp = self.__sendMsg(msg, self.__outgoingsocket)
            if resp[0] != '0':
                print('Error: '+resp[1])
            msg = input(':')
        self.__sendMsg('QUIT|'+phone, self.__outgoingsocket)
        
    def __recvMsgs(self):
        msg = self.__incomingsocket.recv(1024).decode('UTF-8').strip()
        while msg != 'QUIT':
            arguments = msg.strip().split('|')
            print('User '+arguments[1]+' says: '+arguments[4])
            msg = self.__incomingsocket.recv(1024).decode('UTF-8').strip()
        

In [2]:
c = Client(5000)
c.runClient()

 Client Messaging
___________________
1. Login
2. Create User
3. Close


: 1
Phone:  303
Password:  pwd


Error: incorrect username or password
 Client Messaging
___________________
1. Login
2. Create User
3. Close


: 2
Phone:  303 pwd
Password:  pwd


To send msg, use format M|<recipient number>|<msg>
Type QUIT to quit
User 313 says: Hello


: M|313|hello back
: M|313|still there?


Error: user not online


: QUIT


 Client Messaging
___________________
1. Login
2. Create User
3. Close


: 3
